In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /Train.zip" -d "/content/Train"

Streaming output truncated to the last 5000 lines.
 extracting: /content/Train/content/Train/Anomaly/Explosion046_x264_120010.png  
 extracting: /content/Train/content/Train/Anomaly/Robbery141_x264_2750.png  
 extracting: /content/Train/content/Train/Anomaly/Robbery036_x264_920.png  
  inflating: /content/Train/content/Train/Anomaly/Abuse004_x264_8490.png  
 extracting: /content/Train/content/Train/Anomaly/Fighting050_x264_2100.png  
  inflating: /content/Train/content/Train/Anomaly/Burglary025_x264_5300.png  
  inflating: /content/Train/content/Train/Anomaly/RoadAccidents013_x264_1720.png  
 extracting: /content/Train/content/Train/Anomaly/Shoplifting030_x264_6800.png  
 extracting: /content/Train/content/Train/Anomaly/Stealing086_x264_4490.png  
 extracting: /content/Train/content/Train/Anomaly/Abuse032_x264_710.png  
  inflating: /content/Train/content/Train/Anomaly/Stealing100_x264_7000.png  
 extracting: /content/Train/content/Train/Anomaly/Explosion046_x264_58490.png  
 extractin

In [ ]:
!unzip "/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /Test.zip" -d "/content/Test"

Streaming output truncated to the last 5000 lines.
 extracting: /content/Test/content/Test_th/Anomaly/Fighting003_x264_810.png  
 extracting: /content/Test/content/Test_th/Anomaly/Shooting034_x264_1030.png  
 extracting: /content/Test/content/Test_th/Anomaly/Shoplifting034_x264_1100.png  
 extracting: /content/Test/content/Test_th/Anomaly/Shoplifting010_x264_620.png  
  inflating: /content/Test/content/Test_th/Anomaly/Burglary079_x264_2070.png  
 extracting: /content/Test/content/Test_th/Anomaly/Explosion022_x264_2400.png  
 extracting: /content/Test/content/Test_th/Anomaly/Burglary033_x264_1210.png  
  inflating: /content/Test/content/Test_th/Anomaly/Explosion027_x264_730.png  
 extracting: /content/Test/content/Test_th/Anomaly/RoadAccidents001_x264_380.png  
 extracting: /content/Test/content/Test_th/Anomaly/Assault010_x264_12370.png  
  inflating: /content/Test/content/Test_th/Anomaly/Burglary024_x264_3300.png  
 extracting: /content/Test/content/Test_th/Anomaly/Arrest039_x264_12560

In [ ]:
import shutil
shutil.move('/content/Train/content/Train/Anomaly', '/content/Train/Anomaly')

'/content/Train/Anomaly'

In [ ]:
import shutil
shutil.move('/content/Train/content/Train/NormalVideos', '/content/Train/NormalVideos')

'/content/Train/NormalVideos'

In [ ]:
import shutil
shutil.move('/content/Test/content/Test_th/Anomaly', '/content/Test/Anomaly')

'/content/Test/Anomaly'

In [ ]:
import shutil
shutil.move('/content/Test/content/Test_th/NormalVideos', '/content/Test')

'/content/Test/NormalVideos'

In [ ]:
train_dir = "/content/Train"
test_dir = "/content/Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 1
LR =  0.00003
NUM_CLASSES = 2
CLASS_LABELS = ["Normal","Anomaly"]

In [ ]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_fun
                                  )

In [ ]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True ,
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                   )
test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False ,
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = SEED
                                                  )

Found 1266345 images belonging to 3 classes.
Found 111308 images belonging to 3 classes.


In [ ]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(train_generator.classes).count(i) for i in np.unique(train_generator.classes)] ,
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld")
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Train Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
fig = px.bar(x = CLASS_LABELS,
             y = [list(test_generator.classes).count(i) for i in np.unique(test_generator.classes)] ,
             color = np.unique(train_generator.classes) ,
             color_continuous_scale="Emrld")
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Number of Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Test Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                               include_top=False,
                                               weights="imagenet")(inputs)

    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)

    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)

    return classification_output

def define_compile_model():

    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT ,IMG_WIDTH,3))
    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)

    model.compile(optimizer=tf.keras.optimizers.SGD(LR),
                loss='categorical_crossentropy',
                metrics = [tf.keras.metrics.AUC()])

    return model

model = define_compile_model()
clear_output()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ densenet121 (Functional)             │ (None, 2, 2, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classification (Dense)               │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,088,898 (30.86 MB)

 Trainable params: 8,005,250 (30.54 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf

# Load your model
model_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /model/model_mo/model_epoch_01.keras'
model = tf.keras.models.load_model(model_path)

# Set parameters for image processing
IMG_HEIGHT = 64
IMG_WIDTH = 64
CLASS_LABELS = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', "Normal", 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
ANOMALY_CLASSES = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
THRESHOLD = 0.05  # Set threshold for multi-label classification

# Function to preprocess and predict the class of a single image
def predict_class(image_path):
    # Load and preprocess the image
    img = load_img(image_path, color_mode='rgb', target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.densenet.preprocess_input(img_array)
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)[0]  # Extract first (and only) batch

    # Initialize counters for anomalies
    anomaly_count = 0
    predicted_classes = []

    # Output all classes above the confidence threshold and count anomalies
    for i, confidence in enumerate(predictions):
        if confidence >= THRESHOLD:
            class_label = CLASS_LABELS[i]
            predicted_classes.append((class_label, confidence))
            if class_label in ANOMALY_CLASSES:
                anomaly_count += 1
            print(f"{class_label} - {confidence:.2f}")

    # Determine if the image is "Normal" or "Anomaly" based on the count of anomaly classes
    if anomaly_count >= 1:
        print("\nConclusion: Anomaly")
    else:
        print("\nConclusion: Normal ")

# Example usage
image_path = "/content/Test/Anomaly/Abuse028_x264_0.png"  # Replace with the path to your test image
predict_class(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Abuse - 0.30
Arrest - 0.70

Conclusion: Anomaly


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
import os

# Load your model
model_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /model/model_mo/model_epoch_01.keras'
model = tf.keras.models.load_model(model_path)

# Set parameters for image processing
IMG_HEIGHT = 64
IMG_WIDTH = 64
CLASS_LABELS = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', "Normal", 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
ANOMALY_CLASSES = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
THRESHOLD = 0.05  # Set threshold for multi-label classification

# Function to preprocess and predict the class of a single image
def predict_class(image_path):
    # Load and preprocess the image
    img = load_img(image_path, color_mode='rgb', target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.densenet.preprocess_input(img_array)
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)[0]  # Extract first (and only) batch

    # Initialize counters for anomalies
    anomaly_count = 0
    predicted_classes = []

    # Output all classes above the confidence threshold and count anomalies
    for i, confidence in enumerate(predictions):
        if confidence >= THRESHOLD:
            class_label = CLASS_LABELS[i]
            predicted_classes.append((class_label, confidence))
            if class_label in ANOMALY_CLASSES:
                anomaly_count += 1
            print(f"{class_label} - {confidence:.2f}")

    # Determine if the image is "Normal" or "Anomaly" based on the count of anomaly classes
    if anomaly_count >= 2:
        print("\nConclusion: Anomaly")
        return 1  # Anomaly
    else:
        print("\nConclusion: Normal ")
        return 0  # Normal

# Function to process all images in a directory
def process_directory(directory_path):
    total_images = 0
    anomalies = 0

    # List all files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Only process files that are images (you can adjust the extension types if needed)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            total_images += 1
            print(f"Processing {filename}...")
            result = predict_class(file_path)
            if result == 1:  # If it's an anomaly
                anomalies += 1

    print(f"\nTotal images processed: {total_images}")
    print(f"Total anomalies detected: {anomalies}")

# Example usage
directory_path = "/content/Test/Anomaly"  # Replace with the path to your directory
process_directory(directory_path)


Processing Stealing019_x264_4310.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Abuse - 0.13
Arrest - 0.87

Conclusion: Anomaly
Processing RoadAccidents127_x264_2530.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Abuse - 0.11
Arrest - 0.89

Conclusion: Anomaly
Processing Shooting043_x264_1440.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Abuse - 0.57
Arrest - 0.43

Conclusion: Anomaly
Processing Burglary061_x264_2400.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Abuse - 0.15
Arrest - 0.85

Conclusion: Anomaly
Processing Arrest039_x264_1360.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abuse - 0.28
Arrest - 0.72

Conclusion: Anomaly
Processing Arrest030_x264_1020.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abuse - 0.14
Arrest - 0.86

Conclusion: Anomaly
Processing Explosion029_x264_620.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Abuse - 0.28
Arrest - 0.72

Conclusion: Anomaly
Processing Shooting022_x264_2780.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abuse - 0.19
Arrest - 0.81

Conclusion: A

KeyboardInterrupt: 

In [ ]:
import os

def count_images_in_folder(folder_path):
    # Supported image extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}

    # Count images
    image_count = sum(1 for file in os.listdir(folder_path)
                      if os.path.isfile(os.path.join(folder_path, file))
                      and os.path.splitext(file)[1].lower() in image_extensions)

    return image_count

# Example usage
folder_path = '/content/Test/Anomaly'
print(f"Number of images in folder: {count_images_in_folder(folder_path)}")


Number of images in folder: 46356


Normal

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf
import os

# Load your model
model_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /model/model_mo/model_epoch_01.keras'
model = tf.keras.models.load_model(model_path)

# Set parameters for image processing
IMG_HEIGHT = 64
IMG_WIDTH = 64
CLASS_LABELS = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', "Normal", 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
ANOMALY_CLASSES = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
THRESHOLD = 0.05  # Set threshold for multi-label classification

# Function to preprocess and predict the class of a single image
def predict_class(image_path):
    # Load and preprocess the image
    img = load_img(image_path, color_mode='rgb', target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.densenet.preprocess_input(img_array)
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)[0]  # Extract first (and only) batch

    # Initialize counters for anomalies
    anomaly_count = 0
    predicted_classes = []

    # Output all classes above the confidence threshold and count anomalies
    for i, confidence in enumerate(predictions):
        if confidence >= THRESHOLD:
            class_label = CLASS_LABELS[i]
            predicted_classes.append((class_label, confidence))
            if class_label in ANOMALY_CLASSES:
                anomaly_count += 1
            print(f"{class_label} - {confidence:.2f}")

    # Determine if the image is "Normal" or "Anomaly" based on the count of anomaly classes
    if anomaly_count >= 4:
        print("\nConclusion: Anomaly")
        return 1  # Anomaly
    else:
        print("\nConclusion: Normal ")
        return 0  # Normal

# Function to process all images in a directory
def process_directory(directory_path):
    total_images = 0
    anomalies = 0
    normals = 0  # Counter for normal images

    # List all files in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Only process files that are images (you can adjust the extension types if needed)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            total_images += 1
            print(f"Processing {filename}...")
            result = predict_class(file_path)
            if result == 1:  # If it's an anomaly
                anomalies += 1
            else:  # If it's normal
                normals += 1

    print(f"\nTotal images processed: {total_images}")
    print(f"Total anomalies detected: {anomalies}")
    print(f"Total normal images detected: {normals}")

# Example usage
directory_path = "/content/Test/NormalVideos"  # Replace with the path to your directory
process_directory(directory_path)


Processing Normal_Videos_935_x264_1090.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Abuse - 0.41
Arrest - 0.59

Conclusion: Normal 
Processing Normal_Videos_934_x264_230.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Abuse - 0.33
Arrest - 0.67

Conclusion: Normal 
Processing Normal_Videos_938_x264_3250.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abuse - 0.13
Arrest - 0.87

Conclusion: Normal 
Processing Normal_Videos_940_x264_800.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Abuse - 0.19
Arrest - 0.81

Conclusion: Normal 
Processing Normal_Videos_935_x264_21140.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Abuse - 0.28
Arrest - 0.72

Conclusion: Normal 
Processing Normal_Videos_924_x264_50090.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Arrest - 0.96

Conclusion: Normal 
Processing Normal_Videos_702_x264_1810.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Abuse - 0.10
Arrest - 0.90

Conclusion: Normal 
Processing Normal_Videos_365_x264_560.png...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Abuse - 

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# Load your model
model_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /model/model_mo/model_epoch_01.keras'
model = tf.keras.models.load_model(model_path)

# Set parameters for image processing
IMG_HEIGHT = 64
IMG_WIDTH = 64
ANOMALY_CLASSES = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
THRESHOLD = 0.05  # Set threshold for multi-label classification

# Initialize counter for anomaly images
anomaly_images_count = 0

# Function to preprocess and predict the class of a single image
def predict_class(image_path, serial_no):
    global anomaly_images_count

    # Load and preprocess the image
    img = load_img(image_path, color_mode='rgb', target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.densenet.preprocess_input(img_array)
    img_array = img_array / 255.0

    # Make prediction
    predictions = model.predict(img_array)[0]  # Extract first (and only) batch

    # Print serial number and image path
    print(f"{serial_no}. {image_path}")
    anomaly_count = 0  # Count for number of anomaly classes above the threshold
    for i, confidence in enumerate(predictions):
        if confidence >= THRESHOLD:
            class_label = ANOMALY_CLASSES[i]
            print(f"  {class_label} - {confidence:.2f}")
            anomaly_count += 1

    # If more than one anomaly class is predicted, classify the image as normal
    if anomaly_count > 1:
        print("  Conclusion: Normal")
    else:
        print("  Conclusion: Anomaly")
        anomaly_images_count += 1

# Loop through all images in the specified folder
test_folder = "/content/Test/Abuse"
serial_no = 1
for root, _, files in os.walk(test_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            image_path = os.path.join(root, file)
            predict_class(image_path, serial_no)
            serial_no += 1

# Display summary of anomaly images
print("\nSummary:")
print(f"Total number of anomaly images: {anomaly_images_count}")


video visualization

In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
import tensorflow as tf

# Define paths
input_video_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /testing/ANOMALY VIDEO-TEST1.mp4'  # Replace with your video path
output_video_path = '/content/ANALYZED_video.mp4'

# Model and constants
IMG_HEIGHT, IMG_WIDTH = 64, 64
THRESHOLD = 0.00005
CLASS_LABELS = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', "Normal", 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
ANOMALY_CLASSES = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism']
model_path = '/content/drive/Shareddrives/FALCON Data Drive/thanushri/kaggle/Kaggle_anomlay_action_detection/binary /model/model_mo/model_epoch_01.keras'  # Replace with the correct model path
model = load_model(model_path)

# Helper function to preprocess and predict
def predict_class(frame):
    img_array = cv2.resize(frame, (IMG_HEIGHT, IMG_WIDTH))
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.densenet.preprocess_input(img_array)
    predictions = model.predict(img_array)[0]  # Extract first (and only) batch
    print("Predictions:", predictions)

    anomaly_count = sum(1 for i, confidence in enumerate(predictions) if confidence >= THRESHOLD and CLASS_LABELS[i] in ANOMALY_CLASSES)
    return "Anomaly" if anomaly_count >= 1 else "Normal"

# Process the video
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Predict frame class
    label = predict_class(frame)

    # Annotate frame
    color = (0, 255, 0) if label == "Normal" else (0, 0, 255)
    cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Write annotated frame
    out.write(frame)

cap.release()
out.release()

print(f"Processed video saved to: {output_video_path}")


Streaming output truncated to the last 5000 lines.
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predictions: [0. 1.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predi